# Web Scrapping in Python


**TODO pour moi avant le TME**

1.   Faire la version étudiant en enlevant les réponses. Ne garder qu'un code générique pour la carte qu'ils devront adapter



The first objective of this notebook is to discover the `request` and `BeautifulSoup` libraries to crawl a table on a Wikitable page, build a dataframe, and create a map.

*   `request` and [urllib](https://docs.python.org/3/library/urllib.html#module-urllib) for requestion REST API
*   [BeautifulSoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to inspect webpages

Please note that this part serves as an initiation of web scrapping but you will need to learn by yourself to make the project. It is inspired from notebooks published by Galiana, Lino. 2023. Python Pour La Data Science. https://doi.org/10.5281/zenodo.8229676. Exercices target different sources of information, the code needs to be adapted.











In [57]:
!pip install -q lxml

import bs4
import lxml
import pandas
import urllib
import numpy

from urllib import request

## Exercice 1 : Scrap a table on a wikipedia page

We would like to display on a map the location of Summer Olympic Games since 1896. We will use a [Wikipedia page](https://fr.wikipedia.org/wiki/Jeux_olympiques) to scrap the associated table.
Below is the code to extract the content of the page using `request` and display its title using `BeautifulSoup`.

In [2]:
jo = "https://fr.wikipedia.org/wiki/Jeux_olympiques"

request_text = request.urlopen(jo).read()
print(request_text[:1000])
page = bs4.BeautifulSoup(request_text, "lxml")
print(page.find("title"))

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" lang="fr" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Jeux olympiques \xe2\x80\x94 Wikip\xc3\xa9dia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 ve

Our objective here is to extract the different information in the first table "Jeux olympiques d'été" and to build a data frame.

To proceed, you will have to follow these steps:
*   Find the table
*   Collect each line of the table
*   Retrieve the different columns and transform them into text format. Also, use `strip` to format the value into a proper text format (e.g., without useless spaces). Store these lines (formated as a table of columns) in a table.

**Warning: We have two location for 1940. You have therefore to manage this case: identify the attribute indicating this fact and write a code to collect those lines and insert in the previous table in the right place**

* Collect headers of the HTML table
* Build a data frame from the result table and the headers








In [53]:
jo = "https://fr.wikipedia.org/wiki/Jeux_olympiques"
request_text = request.urlopen(jo).read()
page = bs4.BeautifulSoup(request_text, "lxml")
table = page.findAll("table", {"class": "wikitable center"})

# print(table[0].findAll("tr"))
liste = []
lignes = table[0].findAll("tr")


for tr in lignes:
    i = 0
    liste_case = []
    tds = tr.findAll("td")
    if len(tds) == 5:
        annee = tds[0].get_text(strip=True)
        olympiade = tds[1].get_text(strip=True)
        ville = tds[2].get_text(strip=True)
        pays = tds[3].get_text(strip=True)
        continent = tds[3].get_text(strip=True)
        liste.append((annee, olympiade, ville, pays, continent))
    if len(tds) == 3:
        annee, olympiade, _, _, _ = liste[-1]
        if (tds[0].find('i')):
            ville = tds[0].get_text(strip=True)
            pays = tds[1].get_text(strip=True)
            continent = tds[2].get_text(strip=True)
            liste.append((annee, olympiade, ville, pays, continent))
        else:
            annee = tds[0].get_text(strip=True)
            olympiade = tds[1].find('a').get_text(strip=True)
            ville = "tbd"
            pays = "tbd"
            continent = "tbd"
            liste.append((annee, olympiade, ville, pays, continent))

header_cells = lignes[0].find_all('th')
column_names = [th.get_text(strip=True) for th in header_cells]
    
dico = {column_names[0]: [elem[0] for elem in liste],
        column_names[1]: [elem[1] for elem in liste],
        column_names[2]: [elem[2] for elem in liste],
        column_names[3]: [elem[3] for elem in liste],
        column_names[4]: [elem[4] for elem in liste],
       }
df = pandas.DataFrame(data=dico)

print(df)

   Année Olympiade(Nº de l’olympiade[NB 1])         Ville hôte  \
0   1896                                  I         Athènes(1)   
1   1900                                 II           Paris(1)   
2   1904                                III     Saint-Louis(1)   
3   1906             •(Intercalaires[NB 2])         Athènes(2)   
4   1908                                 IV         Londres(1)   
5   1912                                  V       Stockholm(1)   
6   1916                  VI(annulés[NB 3])             Berlin   
7   1920                                VII          Anvers(1)   
8   1924                               VIII           Paris(2)   
9   1928                                 IX       Amsterdam(1)   
10  1932                                  X     Los Angeles(1)   
11  1936                                 XI          Berlin(1)   
12  1940                 XII(annulés[NB 4])              Tokyo   
13  1940                 XII(annulés[NB 4])           Helsinki   
14  1944  

## Exercice 2: Locate places on a map

The objective here is to identify organizer cities on a map.
You will have to code the following steps:
* Collect the URL of each city
* Go the this page using `urllib.request`
* Find the coordinates
* Store cities and coordinates in a data frame

In [72]:
#[student]

jo = "https://fr.wikipedia.org/wiki/Jeux_olympiques"
request_text = request.urlopen(jo).read()
page = bs4.BeautifulSoup(request_text, "lxml")
table = page.findAll("table", {"class": "wikitable center"})

urls = []
lignes = table[0].findAll("tr")


for tr in lignes:
    td = tr.findAll('td')
    if len(td) >= 2 and td[2].find('a'):
        urls.append("https://fr.wikipedia.org" + td[2].find('a')["href"])


for city in urls:
    

['https://fr.wikipedia.org/wiki/Ath%C3%A8nes', 'https://fr.wikipedia.org/wiki/Paris', 'https://fr.wikipedia.org/wiki/Saint-Louis_(Missouri)', 'https://fr.wikipedia.org/wiki/Ath%C3%A8nes', 'https://fr.wikipedia.org/wiki/Londres', 'https://fr.wikipedia.org/wiki/Stockholm', 'https://fr.wikipedia.org/wiki/Berlin', 'https://fr.wikipedia.org/wiki/Anvers', 'https://fr.wikipedia.org/wiki/Paris', 'https://fr.wikipedia.org/wiki/Amsterdam', 'https://fr.wikipedia.org/wiki/Los_Angeles', 'https://fr.wikipedia.org/wiki/Berlin', 'https://fr.wikipedia.org/wiki/Tokyo', 'https://fr.wikipedia.org/wiki/Londres', 'https://fr.wikipedia.org/wiki/Londres', 'https://fr.wikipedia.org/wiki/Helsinki', 'https://fr.wikipedia.org/wiki/Melbourne', 'https://fr.wikipedia.org/wiki/Rome', 'https://fr.wikipedia.org/wiki/Tokyo', 'https://fr.wikipedia.org/wiki/Mexico', 'https://fr.wikipedia.org/wiki/Munich', 'https://fr.wikipedia.org/wiki/Montr%C3%A9al', 'https://fr.wikipedia.org/wiki/Moscou', 'https://fr.wikipedia.org/wiki/

We then transform degree coordinates into numerical coordinates to build a map


In [ ]:
import re

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S', 'O'):
        dd *= -1
    return dd

def parse_dms(dms):
    parts = re.split('[^\d\w]+', dms)
    #potentially needs an adaptation
    #[student]
    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])
    #lng = dms2dd(parts[4], parts[5], parts[6], parts[7])
    return lat

data['latitude'] = data['latitude'].apply(parse_dms)
#data['longitude'] = data['longitude'].apply(parse_dms)
print(data)

The map can be obtain with the following code

In [ ]:
import geopandas as gpd
from pathlib import Path
import folium
from IPython.display import display

gdf = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.longitude, data.latitude))

Path("leaflet").mkdir(parents=True, exist_ok=True)

center = gdf[['latitude', 'longitude']].mean().values.tolist()
sw = gdf[['latitude', 'longitude']].min().values.tolist()
ne = gdf[['latitude', 'longitude']].max().values.tolist()

m = folium.Map(location = center, tiles='openstreetmap')

# I can add marker one by one on the map
for i in range(0,len(gdf)):
    folium.Marker([gdf.iloc[i]['latitude'], gdf.iloc[i]['longitude']], popup=gdf.iloc[i]['name']).add_to(m)

m.fit_bounds([sw, ne])
display(m)

# Project

The objective of this project is to practice all concepts taught in the main lecture. Therefore, you will have to collect data around a thematic, identify a problematic, clean and format the data, provide some exploratory analysis and visualization, build models and evaluate them. You will have also to design dashboard and to storytell the whole pipeline.


The whole project will have two outputs describing the methodology and the results:
* A technical report targeting your datascientist colleagues
* An oral presentation targeting your CEO, chief, client. In this case, we assume that the audience is not specialized in data science. (But you also need to present the methodology to convince them to trust the results).


**Requests:**
* Team work of two people (same group throughout the semester)
* All your work should be stored on a git repo: [tutorial](https://github.com/baskiotisn/2IN013robot2023/blob/d979333fb80c9b6acd9515aaec040943d10d365c/docs/tutoriel_git.pdf)

**Remarks:**
There are also other libraries or issues you'll encounter when collecting data. Some of these are listed below, but don't be shy and interact with a search engine to solve your own issue!
* [Regular expressions](https://docs.python.org/3/howto/regex.html) might be useful!
* [API with authentication](https://www.geeksforgeeks.org/authentication-using-python-requests/)
*   [Selenium](https://selenium-python.readthedocs.io/) when pages are generated wia javascript scripts
* [Playwright](https://playwright.dev/) --> looks easier and more adapted than Selenium
* [Scrapy](https://scrapy.org/) for web crawling / or when you don't know the URL.[Tuto here](https://doc.scrapy.org/en/latest/intro/tutorial.html)
* [Summary of some difficulties in scrapping data from the web](https://www.zenrows.com/blog/web-scraping-challenges#page-structure-changes)

## Your daily task
Collecting data from the web and open data portals. Open data might include csv files, you can use them, but be aware that scraping should be your main acitivity in the dataset gathering.
You can begin to format the data, merge them to build dataframe that would be analyzed next week.